In [94]:
#input of the data 
['ac_month_date', 'agent_name', 'lob',' reporting_line', 'nep', 'nic', 'noc', 'nop_t']

['ac_month_date',
 'agent_name',
 'lob',
 ' reporting_line',
 'nep',
 'nic',
 'noc',
 'nop_t']

# **Agent analysis**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)
# importing all LOB dataset
lob = pd.read_csv(r'C:\Users\ananya.datta\OneDrive - Fairfax Asia Ltd\Desktop\Project 2 - Agent Scorecard\fct_all_lob_ag_acy_mly.csv')
lob.shape

C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_2344\3077444540.py:3: DtypeWarning: Columns (22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  lob = pd.read_csv(r'C:\Users\ananya.datta\OneDrive - Fairfax Asia Ltd\Desktop\Project 2 - Agent Scorecard\fct_all_lob_ag_acy_mly.csv')


(1586809, 49)

In [3]:
lob.ac_month_date.unique()

array(['2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01',
       '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01',
       '2022-12-01', '2023-04-01', '2023-05-01', '2023-06-01',
       '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01',
       '2023-11-01', '2023-12-01', '2024-04-01', '2024-05-01',
       '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01',
       '2024-10-01', '2024-11-01', '2024-12-01', '2025-04-01',
       '2022-01-01', '2022-02-01', '2022-03-01', '2023-01-01',
       '2023-02-01', '2023-03-01', '2024-01-01', '2024-02-01',
       '2024-03-01', '2025-01-01', '2025-02-01', '2025-03-01'],
      dtype=object)

In [98]:
lob['ac_month_date'] = pd.to_datetime(lob['ac_month_date'])
target_dates = pd.to_datetime(['2025-01-01', '2025-02-01'])
twodays = lob[lob['ac_month_date'].isin(target_dates)]
twodays = twodays[['ac_month_date','agent_name','lob','reporting_line','nep','nic','nop_t','noc']]
twodays.shape

(76950, 8)

In [99]:
twodays['lob'] = twodays['lob'].replace({'AVI': 'HUL', 'UNK': 'HUL', 'ZNA': 'HUL'})
twodays.ac_month_date.value_counts()

ac_month_date
2025-01-01    38940
2025-02-01    38010
Name: count, dtype: int64

In [100]:
twodays= twodays[~((twodays['nep'] == 0))]
twodays = twodays[~((twodays['nop_t'] == 0))]
twodays = twodays[twodays['nic']>=0]
twodays = twodays.dropna(subset = ['agent_name','nep'])

In [101]:
# GROUPBY WITH RESPECT TO AC_MONTH_DATE, AGENT_NAME AND LOB and reporting_line
M = twodays.groupby(['ac_month_date','agent_name','lob','reporting_line']).agg({
    'nep': 'sum',          
    'nic': 'sum',          
    'noc': 'sum', 
    'nop_t':'sum' }).reset_index()

In [102]:
def divide(x, y):
    if y == 0:
        return float('nan') 
    return x / y

def minus(x,y):
    return x-y

In [103]:
M['frequency'] =M.apply(lambda row: divide(row['noc'],row['nop_t']), axis=1)
M['severity'] = M.apply(lambda row: divide(row['nic'],row['noc']), axis=1)
M['loss_ratio'] = M.apply(lambda row: divide(row['nic'],row['nep']), axis=1)

In [104]:
# Thresholds for each line of business (lob)
lob_thresholds = {
    'PAC': 0.50,
    'FIR': 0.50,
    'LIA': 0.50,
    'ENG': 0.60,
    'WRK': 0.20,
    'ACC': 0.45,
    'MAR': 0.35,
    'MED': 0.65,
    'BON': 0.10,
    'HUL': 0.65,
    'MOT': 0.75
}

In [105]:
# Weights
WEIGHT_PROFITABILITY = 0.40
WEIGHT_CLAIMS = 0.30
WEIGHT_GWP_GROWTH = 0.30

# Normalize column
def normalize_columns(col):
    min_val, max_val = col.min(), col.max()
    return (col - min_val) / (max_val - min_val) if min_val != max_val else pd.Series([np.nan]*len(col), index=col.index)

# Profitability calculation
def calculate_profitability(data):
    def assign_lob_score(row):
        threshold = lob_thresholds.get(row['lob'], 0.75)
        return threshold - row['loss_ratio']
    
    data['lob_score'] = data.apply(assign_lob_score, axis=1)
    data['Nlob_score'] = normalize_columns(data['lob_score'])
    return data

# Claims score (no penalty)
def calculate_claims(data):
    data['severity'] = data['severity'].fillna(0)
    data['frequency'] = data['frequency'].fillna(0)
    data['claims_score'] = normalize_columns(data['frequency'] + data['severity'])
    return data

def calculate_nep_growth(data):
    data = data.sort_values(by=['agent_name', 'lob', 'ac_month_date'])
    # Shift the 'ac_month_date' column to get the previous month's value (month-based shift)
    data['prev_month'] = data.groupby(['agent_name', 'lob'])['ac_month_date'].shift(1)
    data['prev_nep'] = data.groupby(['agent_name', 'lob'])['nep'].shift(1)
    # Calculate NEP growth as the difference between current and previous NEP
    data['nep_growth'] = ((data['nep'] - data['prev_nep']) / data['prev_nep'].replace(0, np.nan)).fillna(0)
    data['Nnep_growth'] = normalize_columns(data['nep_growth'])
    return data

# Final score
def calculate_final_score(data):
    data['final_score'] = (
        WEIGHT_PROFITABILITY * data['Nlob_score'] +
        WEIGHT_CLAIMS * data['claims_score'] +
        WEIGHT_GWP_GROWTH * data['Nnep_growth']
    )
    return data

# Main pipeline
def process_agent_score(data, lob_thresholds):
    data = calculate_profitability(data)
    data = calculate_claims(data)
    data = calculate_nep_growth(data)
    data = calculate_final_score(data)
    return data

In [106]:
data = M.copy()
data = process_agent_score(data, lob_thresholds)

In [107]:
m = data.copy()
m = m[['ac_month_date', 'agent_name', 'reporting_line','lob','final_score','nep','nic','nop_t','noc', 'lob_score','frequency','severity','nep_growth']]

In [108]:
# GROUPBY WITH RESPECT TO account_yr, AGENT_NAME and getting LOB score
m = m.groupby(['ac_month_date', 'agent_name','reporting_line']).agg({
    'lob': lambda x: ', '.join(sorted(set(x))),  # Unique, sorted lob values per group
    'nep': 'sum',
    'nic': 'sum',
    'noc': 'sum',
    'nop_t': 'sum','final_score':'sum'
}).reset_index()

In [109]:
# Define a function to process the DataFrames
def process_reporting_line_df(df):
    df = df.sort_values(by=['agent_name', 'ac_month_date'], ascending=[True, True]).reset_index(drop=True)
    df['nic/nep ratio'] = df['nic'] / df['nep']
    return df

In [110]:
m = m[m.agent_name != 'THE PACIFIC INSURANCE BERHAD (KL)']
# Create a dictionary of DataFrames keyed by reporting_line values
reporting_line_dfs = {
    line: m[m['reporting_line'] == line].copy()
    for line in m['reporting_line'].unique()
}

In [111]:
# Apply the `process_reporting_line_df` function on all DataFrames in the dictionary
for name, df_subset in reporting_line_dfs.items():
    reporting_line_dfs[name] = process_reporting_line_df(df_subset)

branches_df = reporting_line_dfs['Branches']
pos_affinity_df = reporting_line_dfs['POS/Affinity']
kl_agency_df = reporting_line_dfs['KL Agency']
pamp_affinity_df = reporting_line_dfs['PAMB Affinity']
alt_channel_df = reporting_line_dfs['Alternative Channel']
reinsurance_df = reporting_line_dfs['Reinsurance']
dbcc_df = reporting_line_dfs['DBCC']

In [112]:
def classify_agents(df, score_col='final_score', ratio_col='nic/nep ratio',
                    score_pct=0.75, ratio_pct=0.70):
    """
    Classifies agents into 4 quadrants based on final_score and NIC/NEP ratio.

    Parameters:
    - df: DataFrame containing agent performance data
    - score_col: Column name for final_score
    - ratio_col: Column name for NIC/NEP ratio
    - score_pct: Percentile threshold for high final_score (default = 75th)
    - ratio_pct: Percentile threshold for high NIC/NEP ratio (default = 65th)

    Returns:
    - DataFrame with additional columns: 'score_level', 'ratio_level', 'agent_class'
    """
    # Replace 0s or negatives in ratio with small positive value for safety
    df[ratio_col] = df[ratio_col].clip(lower=0)

    # Calculate thresholds
    score_thresh = df[score_col].quantile(score_pct)
    ratio_thresh = df[ratio_col].quantile(ratio_pct)

    # Assign High/Low levels
    df['score_level'] = np.where(df[score_col] >= score_thresh, 'High', 'Low')
    df['ratio_level'] = np.where(df[ratio_col] >= ratio_thresh, 'High', 'Low')

    # Map quadrant labels
    def assign_class(row):
        if row['score_level'] == 'High' and row['ratio_level'] == 'Low':
            return 'Efficient Performer'
        elif row['score_level'] == 'High' and row['ratio_level'] == 'High':
            return 'Costly Performer'
        elif row['score_level'] == 'Low' and row['ratio_level'] == 'High':
            return 'High-Risk Underperformer'
        else:
            return 'Low-Impact Agent'

    df['agent_class'] = df.apply(assign_class, axis=1)

    return df

In [113]:
def apply_classification_to_all_dfs(reporting_line_dfs):
    """
    Applies the classify_agents function to all DataFrames in the reporting_line_dfs dictionary,
    grouped by 'account_yr' and returning the processed DataFrames.

    Parameters:
    - reporting_line_dfs: Dictionary of DataFrames with reporting lines as keys

    Returns:
    - Dictionary of DataFrames with classification applied
    """
    # Apply classification for each reporting line DataFrame
    for name, df_subset in reporting_line_dfs.items():
        # Group by 'account_yr' and apply classify_agents function
        reporting_line_dfs[name] = df_subset.groupby('ac_month_date').apply(classify_agents).reset_index(drop=True)
        
        # Sort the DataFrame by 'agent_name' and 'account_yr'
        reporting_line_dfs[name] = reporting_line_dfs[name].sort_values(by=['agent_name', 'ac_month_date']).reset_index(drop=True)
        
        # Optional: Print the shape of each processed DataFrame for verification
        print(f"{name} DataFrame shape after classification: {reporting_line_dfs[name].shape}")
    
    return reporting_line_dfs

In [114]:
reporting_line_dfs = apply_classification_to_all_dfs(reporting_line_dfs)
branches_df = reporting_line_dfs['Branches']
pos_affinity_df = reporting_line_dfs['POS/Affinity']
kl_agency_df = reporting_line_dfs['KL Agency']
pamp_affinity_df = reporting_line_dfs['PAMB Affinity']
alt_channel_df = reporting_line_dfs['Alternative Channel']
reinsurance_df = reporting_line_dfs['Reinsurance']
dbcc_df = reporting_line_dfs['DBCC']

Branches DataFrame shape after classification: (2718, 13)
PAMB Affinity DataFrame shape after classification: (289, 13)
KL Agency DataFrame shape after classification: (442, 13)
POS/Affinity DataFrame shape after classification: (583, 13)
Alternative Channel DataFrame shape after classification: (66, 13)
Reinsurance DataFrame shape after classification: (30, 13)
DBCC DataFrame shape after classification: (15, 13)


C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_6552\1136445426.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('ac_month_date').apply(classify_agents).reset_index(drop=True)
C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_6552\1136445426.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('ac_month_date').

In [115]:
finaldf = pd.concat([branches_df, reinsurance_df, dbcc_df, kl_agency_df, pos_affinity_df, pamp_affinity_df, alt_channel_df], ignore_index=True)

In [118]:
finaldf.tail()

,ac_month_date,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
4138,2025-01-01,UOB Kay Hian Wealth Advisory Sdn Bhd,Alternative Channel,"ACC, ENG, FIR, LIA, MED",4895.013654,0.00,0.0,6.0,2.007397,0.000000,High,Low,Efficient Performer
4139,2025-02-01,UOB Kay Hian Wealth Advisory Sdn Bhd,Alternative Channel,PAC,2763.225128,0.00,0.0,4.0,0.401478,0.000000,Low,Low,Low-Impact Agent
4140,2025-01-01,VT Managements Agency,Alternative Channel,"BON, FIR, LIA, MOT, PAC, WRK",59172.364162,188163.72,5.0,61.0,2.426594,3.179926,High,High,Costly Performer
4141,2025-02-01,VT Managements Agency,Alternative Channel,"MED, MOT",46906.109229,141586.49,5.0,33.0,0.816235,3.018509,Low,High,High-Risk Underperformer
4142,2025-01-01,Zul Azreen Enterprise,Alternative Channel,MOT,482.372195,0.00,0.0,1.0,0.401490,0.000000,Low,Low,Low-Impact Agent


# **Pentaho stuff**

### Get data & initial preprocessing

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

In [2]:
import psycopg2
from psycopg2 import sql
 
# Define the database connection parameters
host = "172.16.10.212"
dbname = "PIB_PRD"
user = "usr_app_pdi"
password = 'Usr@ppPd!#123'
port = "5432"  # Default is 5432
 
# Establish the connection
try:
    connection = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password,
        port=port
    )
 
    print("Connected to the database")
 
    # Create a cursor object
    cursor = connection.cursor()
    query = """
    SELECT ac_month_date, agent_name, lob, reporting_line, nep, nic, noc, nop_t 
    FROM ads.fct_all_lob_ag_acy_mly 
    WHERE ac_month_date >= DATE '2021-01-01'"""
    dff = pd.read_sql_query(query, connection)
    print(dff.head())
 
except Exception as e:
    print("Error: Unable to connect to the database")
    print(e)
 
# finally:
#     # Close the cursor and connection
#     if cursor:
#         cursor.close()
#     if connection:
#         connection.close()

Connected to the database


C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_17228\3571967874.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dff = pd.read_sql_query(query, connection)


  ac_month_date                    agent_name  lob     reporting_line  nep  \
0    2022-06-01  M ADVISORY SOLUTIONS SDN BHD  MDG  Alternate Channel  0.0   
1    2022-06-01  M ADVISORY SOLUTIONS SDN BHD  MDG  Alternate Channel  0.0   
2    2022-06-01  M ADVISORY SOLUTIONS SDN BHD  MDG  Alternate Channel  0.0   
3    2022-06-01  M ADVISORY SOLUTIONS SDN BHD  MDG  Alternate Channel  0.0   
4    2022-06-01     Firmus Financial Sdn. Bhd  MDI           Branches  0.0   

   nic  noc  nop_t  
0  0.0  0.0    0.0  
1  0.0  0.0    0.0  
2  0.0  0.0    0.0  
3  0.0  0.0    0.0  
4  0.0  0.0    0.0  


In [3]:
dff.head()

,ac_month_date,agent_name,lob,reporting_line,nep,nic,noc,nop_t
0,2022-06-01,M ADVISORY SOLUTIONS SDN BHD,MDG,Alternate Channel,0.0,0.0,0.0,0.0
1,2022-06-01,M ADVISORY SOLUTIONS SDN BHD,MDG,Alternate Channel,0.0,0.0,0.0,0.0
2,2022-06-01,M ADVISORY SOLUTIONS SDN BHD,MDG,Alternate Channel,0.0,0.0,0.0,0.0
3,2022-06-01,M ADVISORY SOLUTIONS SDN BHD,MDG,Alternate Channel,0.0,0.0,0.0,0.0
4,2022-06-01,Firmus Financial Sdn. Bhd,MDI,Branches,0.0,0.0,0.0,0.0


In [4]:
dff.shape

(3515333, 8)

In [5]:
df = dff.copy()
df.ac_month_date.unique()

array([datetime.date(2022, 6, 1), datetime.date(2022, 7, 1),
       datetime.date(2022, 8, 1), datetime.date(2022, 9, 1),
       datetime.date(2022, 10, 1), datetime.date(2022, 11, 1),
       datetime.date(2022, 12, 1), datetime.date(2023, 6, 1),
       datetime.date(2023, 7, 1), datetime.date(2023, 8, 1),
       datetime.date(2023, 9, 1), datetime.date(2023, 10, 1),
       datetime.date(2023, 11, 1), datetime.date(2023, 12, 1),
       datetime.date(2024, 6, 1), datetime.date(2024, 7, 1),
       datetime.date(2024, 8, 1), datetime.date(2024, 9, 1),
       datetime.date(2024, 10, 1), datetime.date(2024, 11, 1),
       datetime.date(2024, 12, 1), datetime.date(2025, 6, 1),
       datetime.date(2022, 1, 1), datetime.date(2022, 2, 1),
       datetime.date(2022, 3, 1), datetime.date(2022, 4, 1),
       datetime.date(2022, 5, 1), datetime.date(2023, 1, 1),
       datetime.date(2023, 2, 1), datetime.date(2023, 3, 1),
       datetime.date(2023, 4, 1), datetime.date(2023, 5, 1),
       datetime

In [6]:
df['ac_month_date'] = pd.to_datetime(df['ac_month_date'])
df['account_yr'] = df['ac_month_date'].dt.year
df['lob'] = df['lob'].replace({'AVI': 'HUL', 'UNK': 'HUL', 'ZNA': 'HUL'})

In [7]:
df= df[~((df['nep'] == 0))]
df = df[~((df['nop_t'] == 0))]
df = df[df['nic']>=0]
df = df.dropna(subset = ['agent_name','nep'])

In [8]:
df.shape

(382289, 9)

In [9]:
M = df.groupby(['ac_month_date','agent_name','lob','reporting_line']).agg({
    'nep': 'sum',          
    'nic': 'sum',          
    'noc': 'sum', 
    'nop_t':'sum',
    'account_yr':'first'}).reset_index()
print(f'monthly original grouped data: {M.shape}')

M['ac_month_date'] = pd.to_datetime(M['ac_month_date'])
M1 = M[M['ac_month_date'] < '2025-01-01'] 
M2 = M[M['ac_month_date'] >= '2025-01-01']

N1 = M1.groupby(['account_yr','agent_name','lob','reporting_line']).agg({
    'nep': 'sum',          
    'nic': 'sum',          
    'noc': 'sum', 
    'nop_t':'sum'}).reset_index()
print(f'2022-2024 monthly grouped data: {M1.shape}')
print(f'2022-2024 yearwise grouped data: {N1.shape}')
print(f'2025 monthly grouped data: {M2.shape}')

monthly original grouped data: (181941, 9)
2022-2024 monthly grouped data: (161486, 9)
2022-2024 yearwise grouped data: (29246, 8)
2025 monthly grouped data: (20455, 9)


In [10]:
M2.tail()

,ac_month_date,agent_name,lob,reporting_line,nep,nic,noc,nop_t,account_yr
181936,2025-05-01,Ziyou Trading,MOT,Branches,528.550549,0.00,0.0,24.0,2025
181937,2025-05-01,Zulazlee Bin Amin,MOT,Branches,75452.820763,162015.38,2.0,292.0,2025
181938,2025-05-01,Zulkarnain Bin Ismail,MOT,Branches,40322.795356,20690.96,4.0,154.0,2025
181939,2025-05-01,Zum Maju Enterprise,MOT,Branches,13204.398308,0.00,0.0,140.0,2025
181940,2025-05-01,Zurich General Insurance Malaysia Bhd (CAB),PAC,Branches,9673.280963,0.00,0.0,39.0,2025


In [11]:
def divide(x, y):
    if y == 0:
        return float('nan') 
    return x / y

def minus(x,y):
    return x-y

### 2021-24

In [61]:

N1['frequency'] = N1.apply(lambda row: divide(row['noc'],row['nop_t']), axis=1)
N1['severity'] = N1.apply(lambda row: divide(row['nic'],row['noc']), axis=1)
N1['loss_ratio'] = N1.apply(lambda row: divide(row['nic'],row['nep']), axis=1)

In [12]:
# Thresholds for each line of business (lob)
lob_thresholds = {
    'PAC': 0.50,
    'FIR': 0.50,
    'LIA': 0.50,
    'ENG': 0.60,
    'WRK': 0.20,
    'ACC': 0.45,
    'MAR': 0.35,
    'MED': 0.65,
    'BON': 0.10,
    'HUL': 0.65,
    'MOT': 0.75
}
# Weights
WEIGHT_PROFITABILITY = 0.40
WEIGHT_CLAIMS = 0.30
WEIGHT_GWP_GROWTH = 0.30

# Normalize column
def normalize_columns(col):
    min_val, max_val = col.min(), col.max()
    return (col - min_val) / (max_val - min_val) if min_val != max_val else pd.Series([np.nan]*len(col), index=col.index)

# Profitability calculation
def calculate_profitability(data):
    def assign_lob_score(row):
        threshold = lob_thresholds.get(row['lob'], 0.75)
        return threshold - row['loss_ratio']
    
    data['lob_score'] = data.apply(assign_lob_score, axis=1)
    data['Nlob_score'] = normalize_columns(data['lob_score'])
    return data

# Claims score (no penalty)
def calculate_claims(data):
    data['severity'] = data['severity'].fillna(0)
    data['frequency'] = data['frequency'].fillna(0)
    data['claims_score'] = data['frequency'] + data['severity']
    data['Nclaims_score'] = normalize_columns(data['frequency'] + data['severity'])
    return data

# GWP growth calculation
def calculate_nep_growth(data):
    data = data.sort_values(by=['agent_name', 'lob', 'account_yr'])
    data['prev_nep'] = data.groupby(['agent_name', 'lob'])['nep'].shift(1).fillna(0)
    data['nep_growth'] = ((data['nep'] - data['prev_nep']) / data['prev_nep'].replace(0, np.nan)).fillna(0)
    data['Nnep_growth'] = normalize_columns(data['nep_growth'])
    return data

# Final score
def calculate_final_score(data):
    data['final_score'] = (
        WEIGHT_PROFITABILITY * data['Nlob_score'] +
        WEIGHT_CLAIMS * data['Nclaims_score'] +
        WEIGHT_GWP_GROWTH * data['Nnep_growth']
    )
    return data

# Main pipeline
def process_agent_score(data, lob_thresholds):
    data = calculate_profitability(data)
    data = calculate_claims(data)
    data = calculate_nep_growth(data)
    data = calculate_final_score(data)
    return data

In [63]:
data = N1.copy()
data = process_agent_score(data, lob_thresholds)
data = data[['account_yr', 'agent_name', 'reporting_line','lob','final_score','nep','nic','nop_t','noc', 'lob_score','frequency','severity','nep_growth']]
data.shape

(29246, 13)

In [64]:
data.head()

,account_yr,agent_name,reporting_line,lob,final_score,nep,nic,nop_t,noc,lob_score,frequency,severity,nep_growth
9852,2023,365 Unity Sdn Bhd,Branches,FIR,0.032043,2982.739890,0.00,14.0,0.0,0.500000,0.000000,0.00,0.000000
19689,2024,365 Unity Sdn Bhd,Branches,FIR,0.032045,6273.930620,0.00,17.0,0.0,0.500000,0.000000,0.00,1.103412
9853,2023,365 Unity Sdn Bhd,Branches,MOT,0.033841,1521.865534,6464.25,7.0,1.0,-3.497583,0.142857,6464.25,0.000000
19690,2024,365 Unity Sdn Bhd,Branches,MOT,0.032056,2464.293995,0.00,5.0,0.0,0.750000,0.000000,0.00,0.619259
0,2022,A K SEER AGENCY,Branches,ACC,0.032040,504.910495,0.00,4.0,0.0,0.450000,0.000000,0.00,0.000000


In [65]:
# GROUPBY WITH RESPECT TO account_yr, AGENT_NAME and getting LOB score
data = data.groupby(['account_yr', 'agent_name','reporting_line']).agg({
    'lob': lambda x: ', '.join(sorted(set(x))),  # Unique, sorted lob values per group
    'nep': 'sum',
    'nic': 'sum',
    'noc': 'sum',
    'nop_t': 'sum','final_score':'sum'
}).reset_index()
data.shape

(9732, 9)

In [66]:
def process_reporting_line_df(df):
    df = df.sort_values(by=['agent_name', 'account_yr'], ascending=[True, True]).reset_index(drop=True)
    df['nic/nep ratio'] = df['nic'] / df['nep']
    return df

data = data[data.agent_name != 'THE PACIFIC INSURANCE BERHAD (KL)']
# Create a dictionary of DataFrames keyed by reporting_line values
reporting_line_dfs = {
    line: data[data['reporting_line'] == line].copy()
    for line in data['reporting_line'].unique()
}
# Apply the `process_reporting_line_df` function on all DataFrames in the dictionary
for name, df_subset in reporting_line_dfs.items():
    reporting_line_dfs[name] = process_reporting_line_df(df_subset)

In [67]:
branches_df = reporting_line_dfs['Branches']
pos_affinity_df = reporting_line_dfs['POS/Affinity']
kl_agency_df = reporting_line_dfs['KL Agency']
pamp_affinity_df = reporting_line_dfs['PAMB Affinity']
alt_channel_df = reporting_line_dfs['Alternative Channel']
reinsurance_df = reporting_line_dfs['Reinsurance']
dbcc_df = reporting_line_dfs['DBCC']

In [68]:
branches_df[branches_df.account_yr == 2025]

,account_yr,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio


In [13]:
def classify_agents(df, score_col='final_score', ratio_col='nic/nep ratio',
                    score_pct=0.75, ratio_pct=0.70):
    # Replace 0s or negatives in ratio with small positive value for safety
    df[ratio_col] = df[ratio_col].clip(lower=0)

    # Calculate thresholds
    score_thresh = df[score_col].quantile(score_pct)
    ratio_thresh = df[ratio_col].quantile(ratio_pct)

    # Assign High/Low levels
    df['score_level'] = np.where(df[score_col] > score_thresh, 'High', 'Low')
    df['ratio_level'] = np.where(df[ratio_col] > ratio_thresh, 'High', 'Low')

    # Map quadrant labels
    def assign_class(row):
        if row['score_level'] == 'High' and row['ratio_level'] == 'Low':
            return 'Efficient Performer'
        elif row['score_level'] == 'High' and row['ratio_level'] == 'High':
            return 'Costly Performer'
        elif row['score_level'] == 'Low' and row['ratio_level'] == 'High':
            return 'High-Risk Underperformer'
        else:
            return 'Low-Impact Agent'
    df['agent_class'] = df.apply(assign_class, axis=1)
    return df

def apply_classification_to_all_dfs(reporting_line_dfs):
    # Apply classification for each reporting line DataFrame
    for name, df_subset in reporting_line_dfs.items():
        # Group by 'account_yr' and apply classify_agents function
        reporting_line_dfs[name] = df_subset.groupby('account_yr').apply(classify_agents).reset_index(drop=True)
        
        # Sort the DataFrame by 'agent_name' and 'account_yr'
        reporting_line_dfs[name] = reporting_line_dfs[name].sort_values(by=['agent_name', 'account_yr']).reset_index(drop=True)
        
        # Optional: Print the shape of each processed DataFrame for verification
        print(f"{name} DataFrame shape after classification: {reporting_line_dfs[name].shape}")
    
    return reporting_line_dfs

In [70]:
reporting_line_dfs = apply_classification_to_all_dfs(reporting_line_dfs)
branches_df = reporting_line_dfs['Branches']
pos_affinity_df = reporting_line_dfs['POS/Affinity']
kl_agency_df = reporting_line_dfs['KL Agency']
pamp_affinity_df = reporting_line_dfs['PAMB Affinity']
alt_channel_df = reporting_line_dfs['Alternative Channel']
reinsurance_df = reporting_line_dfs['Reinsurance']
dbcc_df = reporting_line_dfs['DBCC']

C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_14592\1333823175.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('account_yr').apply(classify_agents).reset_index(drop=True)
C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_14592\1333823175.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('account_yr').appl

Branches DataFrame shape after classification: (5798, 13)
PAMB Affinity DataFrame shape after classification: (686, 13)
KL Agency DataFrame shape after classification: (927, 13)
POS/Affinity DataFrame shape after classification: (2046, 13)
Reinsurance DataFrame shape after classification: (83, 13)
Alternative Channel DataFrame shape after classification: (139, 13)
DBCC DataFrame shape after classification: (33, 13)


C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_14592\1333823175.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('account_yr').apply(classify_agents).reset_index(drop=True)
C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_14592\1333823175.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('account_yr').appl

In [71]:
finaldf = pd.concat([branches_df, reinsurance_df, dbcc_df, kl_agency_df, pos_affinity_df, pamp_affinity_df, alt_channel_df], ignore_index=True)
finaldf.shape

(9712, 13)

In [72]:
finaldf[finaldf.reporting_line == 'KL Agency']
finaldf[finaldf.agent_name == 'ACP Advance Management']

,account_yr,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
5914,2022,ACP Advance Management,KL Agency,"ACC, FIR, LIA, MED, MOT, PAC",37738.443062,11222.25,3.0,153.666667,0.194901,0.297369,High,Low,Efficient Performer
5915,2023,ACP Advance Management,KL Agency,"ACC, FIR, LIA, MED, MOT, PAC",49660.555775,15523.21,3.0,156.000000,0.193797,0.312586,Low,Low,Low-Impact Agent
5916,2024,ACP Advance Management,KL Agency,"ACC, FIR, LIA, MED, MOT, PAC",64341.814728,50304.70,7.0,180.000000,0.195349,0.781835,High,High,Costly Performer


### 2025

In [14]:
M2.ac_month_date.value_counts()

ac_month_date
2025-04-01    4752
2025-01-01    4510
2025-05-01    4026
2025-02-01    3814
2025-03-01    3353
Name: count, dtype: int64

In [15]:
M2[M2.ac_month_date == '2025-04-01']

,ac_month_date,agent_name,lob,reporting_line,nep,nic,noc,nop_t,account_yr
173163,2025-04-01,A K SEER AGENCY,FIR,Branches,302.654918,0.00,0.0,5.000000,2025
173164,2025-04-01,A K SEER AGENCY,LIA,Branches,36.644615,0.00,0.0,1.333333,2025
173165,2025-04-01,A K SEER AGENCY,MED,Branches,9725.506905,3352.10,1.0,4.000000,2025
173166,2025-04-01,A K SEER AGENCY,MOT,Branches,13136.400637,3226.91,2.0,25.000000,2025
173167,2025-04-01,A K SEER AGENCY,PAC,Branches,287.360357,0.00,0.0,2.666667,2025
...,...,...,...,...,...,...,...,...,...
177910,2025-04-01,Zulazlee Bin Amin,MED,Branches,181.141071,0.00,0.0,1.000000,2025
177911,2025-04-01,Zulazlee Bin Amin,MOT,Branches,45694.205835,29995.02,1.0,164.000000,2025
177912,2025-04-01,Zulkarnain Bin Ismail,MOT,Branches,23877.766694,34037.25,1.0,94.000000,2025
177913,2025-04-01,Zum Maju Enterprise,MOT,Branches,9875.775754,58.50,0.0,75.000000,2025


In [16]:
print(M2.shape)
M2 = M2.groupby(['ac_month_date','agent_name','lob','reporting_line']).agg({
    'nep': 'sum',          
    'nic': 'sum',          
    'noc': 'sum', 
    'nop_t':'sum' }).reset_index()
print(M2.shape)

(20455, 9)
(20455, 8)


In [17]:
M2.head()

,ac_month_date,agent_name,lob,reporting_line,nep,nic,noc,nop_t
0,2025-01-01,365 Unity Sdn Bhd,FIR,Branches,675.040440,0.00,0.0,2.000000
1,2025-01-01,A K SEER AGENCY,MOT,Branches,14664.894451,1462.50,1.0,27.000000
2,2025-01-01,A NAGENDRAN AGENCY,LIA,Branches,19.881472,0.00,0.0,0.333333
3,2025-01-01,A NAGENDRAN AGENCY,MOT,Branches,7721.484663,593.19,0.0,6.000000
4,2025-01-01,A NAGENDRAN AGENCY,PAC,Branches,157.966283,0.00,0.0,0.666667


In [18]:
M2['frequency'] = M2.apply(lambda row: divide(row['noc'],row['nop_t']), axis=1)
M2['severity'] = M2.apply(lambda row: divide(row['nic'],row['noc']), axis=1)
M2['loss_ratio'] = M2.apply(lambda row: divide(row['nic'],row['nep']), axis=1)

In [19]:
def calculate_nep_growth_monthly(data):
    data = data.sort_values(by=['agent_name', 'lob', 'ac_month_date'])
    # Shift the 'ac_month_date' column to get the previous month's value (month-based shift)
    data['prev_month'] = data.groupby(['agent_name', 'lob'])['ac_month_date'].shift(1)
    data['prev_nep'] = data.groupby(['agent_name', 'lob'])['nep'].shift(1)
    # Calculate NEP growth as the difference between current and previous NEP
    data['nep_growth'] = ((data['nep'] - data['prev_nep']) / data['prev_nep'].replace(0, np.nan)).fillna(0)
    data['Nnep_growth'] = normalize_columns(data['nep_growth'])
    return data

# Main pipeline
def process_agent_score_monthly(data, lob_thresholds):
    data = calculate_profitability(data)
    data = calculate_claims(data)
    data = calculate_nep_growth_monthly(data)
    data = calculate_final_score(data)
    return data

In [20]:
data2 = M2.copy()
data2 = process_agent_score_monthly(data2, lob_thresholds)
data2 = data2[['ac_month_date', 'agent_name', 'reporting_line','lob','final_score','nep','nic','nop_t','noc', 'lob_score','frequency','severity','nep_growth']]
data2.shape

(20455, 13)

In [21]:
data2.head()

,ac_month_date,agent_name,reporting_line,lob,final_score,nep,nic,nop_t,noc,lob_score,frequency,severity,nep_growth
0,2025-01-01,365 Unity Sdn Bhd,Branches,FIR,0.410103,675.040440,0.0,2.0,0.0,0.50,0.0,0.0,0.000000
4510,2025-02-01,365 Unity Sdn Bhd,Branches,FIR,0.410011,95.147692,0.0,1.0,0.0,0.50,0.0,0.0,-0.859049
16429,2025-05-01,365 Unity Sdn Bhd,Branches,FIR,0.410242,218.507143,0.0,1.0,0.0,0.50,0.0,0.0,1.296505
4511,2025-02-01,A K SEER AGENCY,Branches,ACC,0.410101,496.139600,0.0,1.0,0.0,0.45,0.0,0.0,0.000000
8324,2025-03-01,A K SEER AGENCY,Branches,ACC,0.410072,364.553297,0.0,1.0,0.0,0.45,0.0,0.0,-0.265220


In [22]:
# GROUPBY WITH RESPECT TO account_yr, AGENT_NAME and getting LOB score
data2 = data2.groupby(['ac_month_date', 'agent_name','reporting_line']).agg({
    'lob': lambda x: ', '.join(sorted(set(x))),  # Unique, sorted lob values per group
    'nep': 'sum',
    'nic': 'sum',
    'noc': 'sum',
    'nop_t': 'sum','final_score':'sum'
}).reset_index()
data2.head()
data2.shape

(9925, 9)

In [23]:
data2 = data2[data2.agent_name != 'THE PACIFIC INSURANCE BERHAD (KL)']
def process_reporting_line_df_monthly(df):
    df = df.sort_values(by=['agent_name', 'ac_month_date'], ascending=[True, True]).reset_index(drop=True)
    df['nic/nep ratio'] = df['nic'] / df['nep']
    return df
reporting_line_dfs = {
    line: data2[data2['reporting_line'] == line].copy()
    for line in data2['reporting_line'].unique()
}
for name, df_subset in reporting_line_dfs.items():
    reporting_line_dfs[name] = process_reporting_line_df_monthly(df_subset)

In [24]:
branches_df = reporting_line_dfs['Branches']
pos_affinity_df = reporting_line_dfs['POS/Affinity']
kl_agency_df = reporting_line_dfs['KL Agency']
pamp_affinity_df = reporting_line_dfs['PAMB Affinity']
alt_channel_df = reporting_line_dfs['Alternative Channel']
reinsurance_df = reporting_line_dfs['Reinsurance']
dbcc_df = reporting_line_dfs['DBCC']

In [25]:
branches_df.shape

(6843, 10)

In [26]:
def apply_classification_to_all_dfs_monthly(reporting_line_dfs):
    # Apply classification for each reporting line DataFrame
    for name, df_subset in reporting_line_dfs.items():
        # Group by 'account_yr' and apply classify_agents function
        reporting_line_dfs[name] = df_subset.groupby('ac_month_date').apply(classify_agents).reset_index(drop=True)
        
        # Sort the DataFrame by 'agent_name' and 'account_yr'
        reporting_line_dfs[name] = reporting_line_dfs[name].sort_values(by=['agent_name', 'ac_month_date']).reset_index(drop=True)
        
        # Optional: Print the shape of each processed DataFrame for verification
        print(f"{name} DataFrame shape after classification: {reporting_line_dfs[name].shape}")
    
    return reporting_line_dfs

In [27]:
reporting_line_dfs = apply_classification_to_all_dfs_monthly(reporting_line_dfs)
branches_df = reporting_line_dfs['Branches']
pos_affinity_df = reporting_line_dfs['POS/Affinity']
kl_agency_df = reporting_line_dfs['KL Agency']
pamp_affinity_df = reporting_line_dfs['PAMB Affinity']
alt_channel_df = reporting_line_dfs['Alternative Channel']
reinsurance_df = reporting_line_dfs['Reinsurance']
dbcc_df = reporting_line_dfs['DBCC']

C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_17228\190407781.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('ac_month_date').apply(classify_agents).reset_index(drop=True)
C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_17228\190407781.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('ac_month_date').ap

Branches DataFrame shape after classification: (6843, 13)
PAMB Affinity DataFrame shape after classification: (730, 13)


C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_17228\190407781.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('ac_month_date').apply(classify_agents).reset_index(drop=True)
C:\Users\ananya.datta\AppData\Local\Temp\ipykernel_17228\190407781.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reporting_line_dfs[name] = df_subset.groupby('ac_month_date').ap

KL Agency DataFrame shape after classification: (1097, 13)
POS/Affinity DataFrame shape after classification: (973, 13)
Alternative Channel DataFrame shape after classification: (153, 13)
Reinsurance DataFrame shape after classification: (67, 13)
DBCC DataFrame shape after classification: (39, 13)


In [28]:
finaldf25 = pd.concat([branches_df, reinsurance_df, dbcc_df, kl_agency_df, pos_affinity_df, pamp_affinity_df, alt_channel_df], ignore_index=True)
finaldf25.shape

(9902, 13)

In [29]:
finaldf25[finaldf25.ac_month_date == '2025-05-01']

,ac_month_date,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
2,2025-05-01,365 Unity Sdn Bhd,Branches,FIR,218.507143,0.00,0.0,1.0,0.410242,0.000000,Low,Low,Low-Impact Agent
7,2025-05-01,A K SEER AGENCY,Branches,"FIR, LIA, MED, MOT, PAC",28410.095322,16555.50,4.0,86.0,2.052428,0.582733,High,High,Costly Performer
12,2025-05-01,A NAGENDRAN AGENCY,Branches,MOT,6023.332967,1521.00,2.0,14.0,0.410464,0.252518,Low,High,High-Risk Underperformer
14,2025-05-01,A.M. Advisory Agency,Branches,MOT,1480.070549,0.00,0.0,16.0,0.410880,0.000000,Low,Low,Low-Impact Agent
19,2025-05-01,AANK Global Enterprise,Branches,MOT,2784.393156,0.00,0.0,26.0,0.410043,0.000000,Low,Low,Low-Impact Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9875,2025-05-01,SLM SDN BHD,Alternative Channel,MOT,167338.132983,9395.10,4.0,86.0,0.411318,0.056144,Low,Low,Low-Impact Agent
9884,2025-05-01,SUMMER CHAIN MARKETING SDN BHD,Alternative Channel,"ACC, BON, FIR, LIA, MOT, PAC",44428.004261,0.00,0.0,292.0,2.460766,0.000000,High,Low,Efficient Performer
9890,2025-05-01,SYNERGY CAP ADVISORY,Alternative Channel,"BON, MED, MOT",28150.536199,0.00,0.0,29.0,1.230339,0.000000,Low,Low,Low-Impact Agent
9895,2025-05-01,UOB Kay Hian Wealth Advisory Sdn Bhd,Alternative Channel,"FIR, LIA, PAC",3482.147940,0.00,0.0,2.0,1.230570,0.000000,High,Low,Efficient Performer


In [33]:
finaldf25.agent_name.unique()

array(['365 Unity Sdn Bhd', 'A K SEER AGENCY', 'A NAGENDRAN AGENCY', ...,
       'UOB Kay Hian Wealth Advisory Sdn Bhd', 'VT Managements Agency',
       'Zul Azreen Enterprise'], dtype=object)

In [34]:
finaldf25[finaldf25.agent_name == 'A K SEER AGENCY']

,ac_month_date,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
3,2025-01-01,A K SEER AGENCY,Branches,MOT,14664.894451,1462.50,1.0,27.0,0.410807,0.099728,Low,Low,Low-Impact Agent
4,2025-02-01,A K SEER AGENCY,Branches,"ACC, FIR, MOT",8693.845688,0.00,0.0,18.0,1.230271,0.000000,High,Low,Efficient Performer
5,2025-03-01,A K SEER AGENCY,Branches,"ACC, LIA, MED, MOT, PAC",8707.015603,0.00,0.0,23.0,2.050484,0.000000,High,Low,Efficient Performer
6,2025-04-01,A K SEER AGENCY,Branches,"FIR, LIA, MED, MOT, PAC",23488.567432,6579.01,3.0,38.0,2.053878,0.280094,High,High,Costly Performer
7,2025-05-01,A K SEER AGENCY,Branches,"FIR, LIA, MED, MOT, PAC",28410.095322,16555.50,4.0,86.0,2.052428,0.582733,High,High,Costly Performer


### Final df [historic & 2025 monthly]

In [91]:
#yearly 2022-24
finaldf.shape

(9712, 13)

In [92]:
#monthly 2025
finaldf25.shape

(9902, 13)

In [93]:
finaldf[finaldf.agent_name == 'Yatida Maju Enterprise']

,account_yr,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
6825,2022,Yatida Maju Enterprise,KL Agency,MOT,262301.712288,65506.59,14.0,1770.0,0.033483,0.249738,Low,Low,Low-Impact Agent
6826,2023,Yatida Maju Enterprise,KL Agency,MOT,273916.117825,498654.81,10.0,2093.0,0.047321,1.820465,Low,High,High-Risk Underperformer
6827,2024,Yatida Maju Enterprise,KL Agency,MOT,217804.318572,322095.88,10.0,1788.0,0.041901,1.478831,Low,High,High-Risk Underperformer


In [94]:
finaldf25[finaldf25.agent_name == 'Yatida Maju Enterprise']

,ac_month_date,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
8028,2025-01-01,Yatida Maju Enterprise,KL Agency,MOT,33877.982764,211.77,0.0,243.0,0.410115,0.006251,Low,Low,Low-Impact Agent
8029,2025-02-01,Yatida Maju Enterprise,KL Agency,MOT,21900.956049,132998.22,1.0,260.0,0.473102,6.072713,Low,High,High-Risk Underperformer
8030,2025-03-01,Yatida Maju Enterprise,KL Agency,MOT,16968.205902,3100.50,1.0,326.0,0.411558,0.182724,Low,High,High-Risk Underperformer
8031,2025-04-01,Yatida Maju Enterprise,KL Agency,MOT,38686.767477,0.00,0.0,511.0,0.410252,0.000000,Low,Low,Low-Impact Agent
8032,2025-05-01,Yatida Maju Enterprise,KL Agency,MOT,62599.312890,176811.96,0.0,904.0,0.410043,2.824503,Low,High,High-Risk Underperformer


In [95]:
finaldf['ac_month_date'] = pd.to_datetime(finaldf['account_yr'].astype(str) + '-01-01')
finaldf['date_type'] = 'YLY'
finaldf = finaldf.drop(['account_yr'], axis=1)
# Build columns without duplicating 'ac_month_date' and 'date_type'
columns = ['ac_month_date', 'date_type'] + [col for col in finaldf.columns if col not in ['ac_month_date', 'date_type']]
finaldf = finaldf[columns]

In [96]:
finaldf.head()

,ac_month_date,date_type,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
0,2023-01-01,YLY,365 Unity Sdn Bhd,Branches,"FIR, MOT",4504.605425,6464.25,1.0,21.000000,0.065884,1.435031,Low,High,High-Risk Underperformer
1,2024-01-01,YLY,365 Unity Sdn Bhd,Branches,"FIR, MOT",8738.224615,0.00,0.0,22.000000,0.064101,0.000000,Low,Low,Low-Impact Agent
2,2022-01-01,YLY,A K SEER AGENCY,Branches,"ACC, ENG, FIR, LIA, MED, MOT, PAC, WRK",229605.445196,115143.20,23.0,456.666667,0.260803,0.501483,High,Low,Efficient Performer
3,2023-01-01,YLY,A K SEER AGENCY,Branches,"ACC, FIR, LIA, MED, MOT, PAC, WRK",182269.127641,363081.96,21.0,357.000000,0.235754,1.992010,High,High,Costly Performer
4,2024-01-01,YLY,A K SEER AGENCY,Branches,"ACC, FIR, LIA, MED, MOT, PAC, WRK",151635.392418,96985.85,13.0,279.000000,0.228059,0.639599,High,Low,Efficient Performer


In [97]:
finaldf25['date_type'] = 'MTD'
# Build columns without duplicating 'ac_month_date' and 'date_type'
columns = ['ac_month_date', 'date_type'] + [col for col in finaldf.columns if col not in ['ac_month_date', 'date_type']]
finaldf25 = finaldf25[columns]

In [98]:
finaldf25.head()

,ac_month_date,date_type,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
0,2025-01-01,MTD,365 Unity Sdn Bhd,Branches,FIR,675.040440,0.0,0.0,2.0,0.410103,0.000000,Low,Low,Low-Impact Agent
1,2025-02-01,MTD,365 Unity Sdn Bhd,Branches,FIR,95.147692,0.0,0.0,1.0,0.410011,0.000000,Low,Low,Low-Impact Agent
2,2025-05-01,MTD,365 Unity Sdn Bhd,Branches,FIR,218.507143,0.0,0.0,1.0,0.410242,0.000000,Low,Low,Low-Impact Agent
3,2025-01-01,MTD,A K SEER AGENCY,Branches,MOT,14664.894451,1462.5,1.0,27.0,0.410807,0.099728,Low,Low,Low-Impact Agent
4,2025-02-01,MTD,A K SEER AGENCY,Branches,"ACC, FIR, MOT",8693.845688,0.0,0.0,18.0,1.230271,0.000000,High,Low,Efficient Performer


In [99]:
# # Reset the index of both dataframes before concatenating
# finaldf_reset = finaldf.reset_index(drop=True)
# finaldf25_reset = finaldf25.reset_index(drop=True)

# Concatenate the dataframes
FINALDF = pd.concat([finaldf, finaldf25], ignore_index=True)

FINALDF.shape

(19614, 14)

In [100]:
FINALDF.columns.to_list()

['ac_month_date',
 'date_type',
 'agent_name',
 'reporting_line',
 'lob',
 'nep',
 'nic',
 'noc',
 'nop_t',
 'final_score',
 'nic/nep ratio',
 'score_level',
 'ratio_level',
 'agent_class']

In [141]:
pip install sqlalchemy

In [114]:
from sqlalchemy import create_engine

# Define the database connection parameters
host = "172.16.10.212"
dbname = "PIB_PRD"
user = "usr_app_pdi"
password = 'Usr@ppPd!#123'
port = "5432"  # Default is 5432


from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Safely encode credentials
safe_user = quote_plus(user)
safe_password = quote_plus(password)

# Build connection string
connection_string = f'postgresql+psycopg2://{safe_user}:{safe_password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)

# # Define your database connection string
# engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Push FINALDF to a table
FINALDF.to_sql('all_lob_ag_class', engine, schema='ads', if_exists='replace', index=False)

print("Dataframe uploaded successfully.")

Both DataFrames uploaded successfully.


In [106]:
# Close the cursor and connection
if cursor:
    cursor.close()
if connection:
    connection.close()

In [101]:
FINALDF.date_type.value_counts()

date_type
MTD    9902
YLY    9712
Name: count, dtype: int64

In [102]:
FINALDF[FINALDF.agent_name == 'A K SEER AGENCY']

,ac_month_date,date_type,agent_name,reporting_line,lob,nep,nic,noc,nop_t,final_score,nic/nep ratio,score_level,ratio_level,agent_class
2,2022-01-01,YLY,A K SEER AGENCY,Branches,"ACC, ENG, FIR, LIA, MED, MOT, PAC, WRK",229605.445196,115143.20,23.0,456.666667,0.260803,0.501483,High,Low,Efficient Performer
3,2023-01-01,YLY,A K SEER AGENCY,Branches,"ACC, FIR, LIA, MED, MOT, PAC, WRK",182269.127641,363081.96,21.0,357.000000,0.235754,1.992010,High,High,Costly Performer
4,2024-01-01,YLY,A K SEER AGENCY,Branches,"ACC, FIR, LIA, MED, MOT, PAC, WRK",151635.392418,96985.85,13.0,279.000000,0.228059,0.639599,High,Low,Efficient Performer
9715,2025-01-01,MTD,A K SEER AGENCY,Branches,MOT,14664.894451,1462.50,1.0,27.000000,0.410807,0.099728,Low,Low,Low-Impact Agent
9716,2025-02-01,MTD,A K SEER AGENCY,Branches,"ACC, FIR, MOT",8693.845688,0.00,0.0,18.000000,1.230271,0.000000,High,Low,Efficient Performer
9717,2025-03-01,MTD,A K SEER AGENCY,Branches,"ACC, LIA, MED, MOT, PAC",8707.015603,0.00,0.0,23.000000,2.050484,0.000000,High,Low,Efficient Performer
9718,2025-04-01,MTD,A K SEER AGENCY,Branches,"FIR, LIA, MED, MOT, PAC",23488.567432,6579.01,3.0,38.000000,2.053878,0.280094,High,High,Costly Performer
9719,2025-05-01,MTD,A K SEER AGENCY,Branches,"FIR, LIA, MED, MOT, PAC",28410.095322,16555.50,4.0,86.000000,2.052428,0.582733,High,High,Costly Performer


In [105]:
df['ac_month_date'] = pd.to_datetime(df['ac_month_date'])
df[(df.agent_name == 'A K SEER AGENCY') & (df.ac_month_date == pd.Timestamp('2025-05-01'))]

,ac_month_date,agent_name,lob,reporting_line,nep,nic,noc,nop_t,account_yr
1442886,2025-05-01,A K SEER AGENCY,MED,Branches,223.955357,0.0,0.0,1.000000,2025
1442888,2025-05-01,A K SEER AGENCY,MED,Branches,3839.806607,0.0,0.0,4.000000,2025
1442889,2025-05-01,A K SEER AGENCY,MED,Branches,2637.865179,0.0,0.0,6.000000,2025
1446830,2025-05-01,A K SEER AGENCY,FIR,Branches,172.335934,0.0,0.0,1.000000,2025
1446834,2025-05-01,A K SEER AGENCY,LIA,Branches,23.384945,0.0,0.0,0.666667,2025
1446837,2025-05-01,A K SEER AGENCY,PAC,Branches,118.125000,0.0,0.0,21.000000,2025
1446840,2025-05-01,A K SEER AGENCY,PAC,Branches,173.725165,0.0,0.0,0.666667,2025
1446841,2025-05-01,A K SEER AGENCY,PAC,Branches,7.142857,0.0,0.0,0.666667,2025
1446842,2025-05-01,A K SEER AGENCY,PAC,Branches,24.107143,0.0,0.0,1.000000,2025
1466206,2025-05-01,A K SEER AGENCY,MOT,Branches,1972.366232,0.0,0.0,2.000000,2025
